# Modelo Preditivo de Crédito

**Autor:** Leonardo Dantas

## Motivação

Técnicas de aprendizado de máquina representam uma ferramenta poderosa na tomada de decisão. Neste caderno interativo, consideraremos como elaborar um modelo de inadimplência para aplicações em cenários de concessão de crédito.

Você pode carregar a formatação deste caderno executando a célula de código a seguir. Toda a formatação é baseada na formatação do curso [Practical Numerical Methods with Python](https://github.com/numerical-mooc/numerical-mooc), licenciado sob a MIT license (c) 2014 L.A. Barba, C. Cooper, G.F. Forsyth, A. Krishnan. O código da análise é licenciado sob a GNU General Public License v3.0.

In [1]:
from IPython.core.display import HTML
from os import getcwd
css_file = f'{getcwd()}/styles/numericalmoocstyle.css'
HTML(open(css_file, 'r').read())

## Modelagem

### Bibliotecas

Toda a análise a seguir será construída em Python. Informações sobre requisitos e versões estão contidas no arquivo `requirements.txt`.

In [2]:
# Geral
from os import getcwd

# Manuseio de Dados
import numpy as np
import pandas as pd

# Aprendizado de Máquina
from sklearn.utils import resample
from sklearn.preprocessing import binarize
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
import xgboost as xgb

### Dados

Como passo inicial, importamos os dados através do Pandas.

In [3]:
raw_train_data = pd.read_csv(f'{getcwd()}/data/treino.csv')

#### Verificação de Variáveis

Antes de manipular os dados, é importante inspecionar as variáveis no conjunto de dados fornecido.

In [4]:
raw_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110000 entries, 0 to 109999
Data columns (total 11 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   inadimplente                           110000 non-null  int64  
 1   util_linhas_inseguras                  110000 non-null  float64
 2   idade                                  110000 non-null  int64  
 3   vezes_passou_de_30_59_dias             110000 non-null  int64  
 4   razao_debito                           110000 non-null  float64
 5   salario_mensal                         88237 non-null   float64
 6   numero_linhas_crdto_aberto             110000 non-null  int64  
 7   numero_vezes_passou_90_dias            110000 non-null  int64  
 8   numero_emprestimos_imobiliarios        110000 non-null  int64  
 9   numero_de_vezes_que_passou_60_89_dias  110000 non-null  int64  
 10  numero_de_dependentes                  107122 non-null  

In [5]:
raw_train_data.head()

,inadimplente,util_linhas_inseguras,idade,vezes_passou_de_30_59_dias,razao_debito,salario_mensal,numero_linhas_crdto_aberto,numero_vezes_passou_90_dias,numero_emprestimos_imobiliarios,numero_de_vezes_que_passou_60_89_dias,numero_de_dependentes
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


Verificamos que todos os dados possuem uma tipagem compatível com sua interpretação. Além disso, identificamos a coluna `inadimplente` como a variável de predição.

#### Análise Preliminar

Em seguida, verificamos a ausência de dados.

In [6]:
raw_train_data.isnull().sum()

inadimplente                                 0
util_linhas_inseguras                        0
idade                                        0
vezes_passou_de_30_59_dias                   0
razao_debito                                 0
salario_mensal                           21763
numero_linhas_crdto_aberto                   0
numero_vezes_passou_90_dias                  0
numero_emprestimos_imobiliarios              0
numero_de_vezes_que_passou_60_89_dias        0
numero_de_dependentes                     2878
dtype: int64

Observe que:

* Apenas 2,5% dos registros não possuem número de dependentes.
* Quase um quinto dos registros não possuem dados sobre salário mensal.

A ausência do número de dependentes não é significante. No entanto, dados sobre salário mensal são de alta relevância para análise de crédito. Como a remoção desses registros resultaria em um desfalque significante para o conjunto, além de determinados modelos conseguirem lidar com dados nulos, prosseguiremos adiante sem alterações, mas cientes dessa característica do conjunto.

Por fim, consideramos o equilíbrio da variável dependente.

In [7]:
non_defaulting = raw_train_data["inadimplente"] == 0
defaulting = raw_train_data["inadimplente"] == 1

defaulting_count = raw_train_data[defaulting].shape[0]
non_defaulting_count = raw_train_data[non_defaulting].shape[0]
total = raw_train_data.shape[0]

defaulting_perc = ( defaulting_count / total ) * 100
non_defaulting_perc = ( non_defaulting_count / total ) * 100

print(f"Inadimplentes: {defaulting_count} | {defaulting_perc}%")
print(f"Adimplentes: {non_defaulting_count} | {non_defaulting_perc}%")

Inadimplentes: 7331 | 6.664545454545455%
Adimplentes: 102669 | 93.33545454545454%


Os dados estão altamente desequilibrados. Devido ao tamanho da discrepância, não seria sensato descartar dados para equilibrar o conjunto. Além disso, há uma possibilidade que tais proporções reflitam o contexto cotidiano de análise de crédito. Cientes dessa característica, retornaremos a essa questão na próxima seção.

#### Partição dos Dados

Em seguida, separamos os dados em treino e teste.

In [8]:
dependent_variable = "inadimplente"
Y = pd.DataFrame(raw_train_data[dependent_variable])
X = raw_train_data.drop(dependent_variable, axis=1)

Particionamos o conjunto original de treino em três subconjuntos:

* **Treino (80%) –** Usado para construir o modelo.
* **Validação (10%) –** Usado para guiar o desenvolvimento do modelo.
* **Teste (10%) –** Usado para avaliar o modelo e possivelmente compará-lo a outros.

Tal partição deve ser feita com cautela. Consideramos que:

1. Há ausência de dados sobre salário mensal.
2. Há uma discrepância de inadimplentes para adimplentes.

Como essas características podem refletir uma realidade do cotidiano do modelo, é oportuno mantê-las ao menos no conjunto de teste. Nesse sentido, estratificaremos os dados de modo que os subconjuntos de teste e treino-validação mantenham as mesmas proporções do conjunto original com relação a *inadimplentes* e *dados faltantes sobre salário*.

Além disso, buscamos resguardar a reprodutibilidade da análise fixando sementes para o embaralhamento de dados.

In [9]:
train_val_test_seed = 56384
test_size= 0.1

X_train_val, X_test, Y_train_val, Y_test = train_test_split(
    X,
    Y,
    test_size=test_size, 
    random_state=train_val_test_seed,
    stratify=pd.concat( 
        [
            Y[dependent_variable], 
            X["salario_mensal"].isnull()
        ], 
        axis= 1
    ) 
)

A partir do conjunto de treino-validação, criaremos os conjuntos de treino e validação propriamente ditos.

Para esses novos subconjuntos, é proveitoso seguir outra abordagem. Como desequilíbrio de dados prejudica diversas técnicas de aprendizado de máquina, é sensato balancear os dados nesses conjuntos através de *upsampling*. Será possível monitorar discrepâncias com um cenário de desequilíbrio através do conjunto de teste.

In [10]:
upsampling_seed = 44890

train_data = pd.concat(
    [
        X_train_val, 
        Y_train_val
    ], 
    axis=1
)

majority_class = train_data[train_data[dependent_variable] == 0]
minority_class = train_data[train_data[dependent_variable] == 1]

upsampled_data = resample(minority_class,
                          replace=True,
                          n_samples=majority_class.shape[0],
                          random_state=upsampling_seed
                         )

balanced_train_data = pd.concat([majority_class, upsampled_data])

assert(majority_class.shape[0] > minority_class.shape[0])

In [11]:
Y_train_val_balanced = pd.DataFrame(
    balanced_train_data[dependent_variable]
)
X_train_val_balanced = balanced_train_data.drop(
    dependent_variable, 
    axis=1
)

In [12]:
train_val_seed = 23745
val_size= 0.1

X_train, X_val, Y_train, Y_val = train_test_split(
    X_train_val_balanced,
    Y_train_val_balanced,
    test_size=val_size, 
    random_state=train_val_seed,
)

Com isso aumentamos a quantidade de dados significantemente.

In [13]:
print("Aprimoramento de Dados de Treino e Validação")
print("Registros Antes: ", X_train_val.shape[0])
print("Registros Depois: ", X_train_val_balanced.shape[0])

Aprimoramento de Dados de Treino e Validação
Registros Antes:  99000
Registros Depois:  184804


### Modelo

Certos princípios guiarão nossa busca por modelos:

* **Simplicidade:** Quanto menos coisas puderem dar errado, melhor. Modelos simples são também tipicamente menos intensos computacionalmente.
* **Interpretabilidade:** Explicar por que um modelo obteve determinados resultados é valioso.
* **Eficácia:** Nada vale se não alcançarmos nosso objetivo. Alto desempenho é almejado.

A fim de atender a essas características, uma classe promissora de modelos são os modelos baseados em árvores, em particular a implementação fornecida na biblioteca XGBoost.

Além disso, o XGBoost possui duas características adicionais úteis:

1. Capacidade de lidar com valores faltantes.
2. Escalabilidade computacional. 

Embora o segundo fator não pareça tão presente na análise, esse aspecto é útil para um eventual aprimoramento ou expansão do modelo.

#### Treino

Como nosso objetivo é determinar a probabilidade de inadimplência, construíremos uma *gradient boosting machine* regressora.

In [14]:
basic_gbm_seed = 32936

basic_gbm = xgb.XGBRegressor(
    objective="binary:logistic",
    learning_rate= 0.1,
    max_depth= 4,
    n_estimators= 5000,
    random_state= basic_gbm_seed,
    verbosity=1,
)

Ajustamos então `basic_gbm` aos dados.

In [15]:
basic_gbm.fit(X_train, Y_train)

[15:10:11] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=4,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=5000, n_jobs=6, num_parallel_tree=1,
             objective='binary:logistic', random_state=32936, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=1)

Com o modelo treinado, interpretamos iterativamente os resultados de treino e validação para fazer ajustes nos parâmetros do modelo.

In [16]:
train_predictions = basic_gbm.predict(X_train)
train_error = mean_squared_error(Y_train, train_predictions)

threshold = 0.5
binary_train_predictions = np.where(train_predictions > 0.5, 1, 0)
binary_train_accuracy = accuracy_score(binary_train_predictions, Y_train)

print(f"Erro de Treino: {train_error}")
print(f"Acurácia de Treino (Binária): {binary_train_accuracy}")

Erro de Treino: 0.04216762050409951
Acurácia de Treino (Binária): 0.9614725564113201


In [17]:
val_predictions = basic_gbm.predict(X_val)
val_error = mean_squared_error(Y_val, val_predictions)

threshold = 0.5
binary_val_predictions = np.where(val_predictions > 0.5, 1, 0)
binary_val_accuracy = accuracy_score(binary_val_predictions, Y_val)

print(f"Erro de Validação: {val_error}")
print(f'Acurácia de Validação (Binária): {binary_val_accuracy}')

Erro de Validação: 0.05460931068653391
Acurácia de Validação (Binária): 0.9432931118445972


Embora uma acurácia de treino alta seja importante, focamos em reduzir a discrepância entre o erro de treino e validação. Queremos evitar o sobreajuste e garantir poder de generalização.

#### Avaliação

Por fim, avaliamos nosso modelo em dados de teste as características originais.

In [18]:
test_predictions = basic_gbm.predict(X_test)
test_error = mean_squared_error(Y_test, test_predictions)

threshold = 0.5
binary_test_predictions = np.where(test_predictions > 0.5, 1, 0)
binary_test_accuracy = accuracy_score(binary_test_predictions, Y_test)

print(f"Erro de Teste: {test_error}")
print(f'Acurácia de Teste (Binária): {binary_test_accuracy}')

Erro de Teste: 0.08991201012624242
Acurácia de Teste (Binária): 0.8774545454545455


Note que a transição de validação para teste tipicamente resulta em uma perda de desempenho. No geral, nosso modelo não sofre uma queda particularmente brusca, mas os resultados podem ser significantes a depender da distribuição de adimplentes para inadimplentes no mundo real. Esse cenário exigiria modelos de altíssima acurácia.

#### Entrega

Por fim geramos as predições para os dados de entrega.

In [19]:
raw_test_data = pd.read_csv(f'{getcwd()}/data/teste.csv')

In [20]:
raw_test_data["inadimplente"] = basic_gbm.predict(raw_test_data)

In [21]:
raw_test_data.to_csv(f'{getcwd()}/data/entrega.csv')

## Conclusão

Com isso chegamos ao fim desta análise. Consideramos as características do conjunto de dados disponível e como técnicas de boosting podem ser utilizadas para análise crédito. Alguns aprimoramentos interessantes para o futuro são:

* **Busca sistemática por hiperparâmetros:** Amostragem de valores possíveis para cada hiperparâmetro com contabilização do erro, se possível usando os conjuntos de treino e validação unidos em validação cruzada.
* **Comparação com outros modelos:** Por exemplo, comparação com modelos baseados em regressão linear e redes neurais artificias (um modelo caixa preta, mas frequentemente bem eficaz).

Há um ramo incrivelmente rico em modelos baseados em árvores que não deixa uma falta de aprimoramentos para o futuro.